In [1]:
import re
import os
import numpy as np
import glob
from tqdm import tqdm
import pandas as pd
from subprocess import Popen, PIPE, call, check_call
import datetime
import shutil
from datetime import datetime
import time

from help_functions import * #user def
from lut import tests



## General defines, paths etc

In [2]:
transcript_path = os.environ['NOELV']
grlib_path = os.environ['GRLIB']
riscv_dv = os.environ['r_dv']+"/"

test_number = 5
print("Running the test: {}".format(tests[test_number]))
now = datetime.now()
current_time = now.strftime("%Y-%m-%d")
finish_after = int(1e5)
boot_loader_length = 41
instr_cnt = finish_after+boot_loader_length+2 #+41 to account for bootloader, +2 for counter trailing

Running the test: riscv_mmu_stress_test


In [31]:
#Each level correponds to one cell
settings = {
    "gen_test":{
        "errors":["You do not have a valid license to run Riviera-PRO"],
        "paths":{
            "test":riscv_dv+"out_"+current_time+"/ovpsim_sim/"+tests[test_number]+"*.log",
            "Makefile":grlib_path+"/designs/noelv-generic/Makefile",
        },
        "test_param" : "python3 run.py --custom_target target/rv64_noelv/ --iss ovpsim --simulator riviera --isa rv64gc --mabi lp64 --test "+tests[test_number]+" --iterations 1",
        "process_param" : 'cd '+riscv_dv+' && source ~/.bashrc && module load aldec/riviera/2021.10 &&',
        "timeout" : False,
        "arg_timeout" : "-gdisas=1 -gfinish_after_en=1 -gfinish_after="+str(instr_cnt),
        "arg_no_timeout" : "-gdisas=1 -gfinish_after_en=0 -gfinish_after=0",
        "iteration":-1, #will be set when run
    },
    "run_rtl":{
        "paths":{
            "cp_dest":transcript_path+'ram.srec',
            "rtl_log":riscv_dv+"out_"+current_time+"/rtl_log/",
            "rtl_log_out":riscv_dv+"out_"+current_time+"/rtl_log/"+tests[test_number],
            "transcript":transcript_path+"transcript",
            
        },
        "test":riscv_dv+"out_"+current_time+"/asm_test/"+tests[test_number]+"*.srec",
        "process_param":'cd '+transcript_path+' && source ~/.bashrc && module add mentor/questasim/2021.3 && make sim-run',
        "test_sucess":["# ** Failure: *** IU in error mode, simulation halted ***","Test finished after","** Failure: Assertion violation."],
    },
    "parse_rtl":{
        "reg_ex":{
            "lookup":re.compile("#\s*\d*\s*ns\s*:\sC\d I\d : \d*\s*\[\d\] @0x[0-9a-fA-F]{16} \(0x[0-9a-fA-F]{4,8}\)\s*.*"),
            "pc_csr_status_tval":re.compile("[0-9a-fA-F]{16}|X{16}"),
            "binary" : re.compile("(?<=(\(0x))[a0-z9]{4,8}(?=\))"), #opcode
            "instruction" : re.compile("\w{2,8}(?=(\s\w{2,3},))|(\w*\.[a-z])|nop|mret|ecall|fence|unknown instruction|ebreak|fld"),
            "instr_str" : re.compile("\w*\s\w*,\s\w*,\s-?\w*(?=\s*W)|\w*(?=\s*W)|\w*\s\w*,\s\w*(?=\s*W)"), #ugly
            "gpr" : re.compile("(?<=([a-z]{2}\s))\w{2,3}(?=,)"),
            "op1" : re.compile("(?<=,\s)\w{1,8}((?=,)|(?=\s*W))|0\(\w{2,3}\)"), 
            "op2" : re.compile("(\w*,\s\w*,\s-?\w*)"), #use split to get op2, don't have a nicer regex atm
            "csr" : re.compile("\w*(?=\s*=\w*\]\[\d\]\sIPC)"),
            "mode" : re.compile("(?<=PRV\[)\d*"),
            "valid" : re.compile("(?<=\[)\d(?=\]\s@)"),
            "exception" : re.compile("(?<=\[)\d(?=\]\sPRV)"),
        },
        "columns":['pc','instr','gpr','csr','binary','mode','instr_str','operand','valid','exception'],

    },
    "gen_csv":{
        "paths":{
            "csv_log":riscv_dv+"out_"+current_time+"/ovpsim_sim/"+tests[test_number]+"*.log",
            "csv_out":riscv_dv+"out_"+current_time+"/ovpsim_sim/"+tests[test_number]+"*.csv",  
        },
        "process":"python3 "+riscv_dv+"scripts/ovpsim_log_to_trace_csv.py --dont_truncate_after_first_ecall --log ",

    },
    "gen_cov":{
        "paths":{
            "logs": riscv_dv+"out_"+current_time+"/ovpsim_sim/",
        },
        "test_param" : "python3 cov.py --iss ovpsim -si riviera --custom_target target/rv64_noelv/",
        "process_param": 'cd '+riscv_dv+' && source ~/.bashrc && module load aldec/riviera/2021.10 && ',
        "test_sucess": "Coverage results are saved",
    }

    
}

## Generate RISCV-DV tests and run OVPsim
- Tests that work: 0
- Tests that sometimes timeout but sometimes doesn't: 1, 2, 10
- Tests that doesn't run at all (sometimes): 1, 2, 5, 6, 8, 9 (Never enters program after running the bootloader)
- Test 7 seems to outright fail

In [33]:
process = Popen(settings['gen_test']['process_param']+settings["gen_test"]["test_param"], shell=True, stdout=PIPE, stderr=PIPE)

stdout, stderr = process.communicate()
error = stderr.decode('ascii')

if(find_error(settings['gen_test']['errors'], error)):
    path = glob.glob(settings['gen_test']['paths']["test"])
    #print(path[0])    
    with open(path[0],"r") as f:
        settings['gen_test']['timeout'] = f.read().find("Info "+str(finish_after)+":") != -1
        f.close()
        
arg = settings["gen_test"]["arg_timeout"] if settings["gen_test"]["timeout"] else settings["gen_test"]["arg_no_timeout"]

with open(settings['gen_test']['paths']['Makefile'],"r") as f:
    txt = f.read()
    tmp = re.sub("(?<=VSIMOPT=).*",arg,txt)
    f.close()
    f = open(settings['gen_test']['paths']['Makefile'],"w")
    f.write(tmp)
    f.close()
    

print("OVPsim timeout = {}".format(settings["gen_test"]["timeout"]))
srec_tests = glob.glob(settings['run_rtl']['test'])
settings['gen_test']['iteration'] = re.search("\d*(?=.srec)",str(srec_tests)).group(0)
print("Program size = "+str(os.path.getsize(srec_tests[0])))


OVPsim timeout = True
Program size = 907148


## Run RTL simulation


In [32]:
srec_tests = glob.glob(settings['run_rtl']['test'])
shutil.copy(srec_tests[0], settings['run_rtl']['paths']['cp_dest']) #copy the current test to the sim dir
print("Copied {} to the NOELV directory".format(srec_tests[0]))
#launch sim
process = Popen(settings['run_rtl']['process_param'], shell=True, stdout=PIPE, stderr=PIPE)
stdout, stderr = process.communicate()
rtl_out = stdout.decode('ascii')


if(find_success(settings['run_rtl']['test_sucess'],rtl_out)):
    print("RTL Simulation Success")
else:
    print("RTL Simulation Failed")

if not os.path.exists(settings['run_rtl']['paths']['rtl_log']):
    os.mkdir(settings['run_rtl']['paths']['rtl_log'])
shutil.copy(settings['run_rtl']['paths']['transcript'], settings['run_rtl']['paths']['rtl_log_out']+"_"+settings['gen_test']['iterations']+".rtl.log")

Copied /home/jonathanjonsson/MasterThesis/riscv-dv/out_2022-03-02/asm_test/riscv_mmu_stress_test_0.srec to the NOELV directory


KeyboardInterrupt: 

## Parse RTL transcript and create rtl_log

In [4]:
match = 0
f = open(settings["run_rtl"]["paths"]["transcript"])
transcript = f.readlines()
f.close()

pc,binary,instr,instr_str,gpr,csr,gpr_val,op_0,op_1,status,tval,mode,exception,valid = [],[],[],[],[],[],[],[],[],[],[],[],[],[]

for line in tqdm(transcript):
    if(re.search(settings["parse_rtl"]["reg_ex"]["lookup"],line)): #check that line conforms to format then extract information
        valid.append(re.search(settings['parse_rtl']['reg_ex']['valid'], line).group(0))
                
        pc_i = re.search(settings['parse_rtl']['reg_ex']["pc_csr_status_tval"], line)
        if(pc_i):
            pc.append(pc_i.group(0))
        else:
            print(line)
            break
        binary.append(re.search(settings['parse_rtl']['reg_ex']["binary"], line).group(0))
        instruction = re.search(settings['parse_rtl']['reg_ex']['instruction'], line)
        if(instruction):
            instr.append(instruction.group(0))
        else:
            raise RuntimeError("Instruction not present in RegEx\n"+line)
        instr_str.append(re.search(settings['parse_rtl']['reg_ex']["instr_str"], line).group(0))
        match3 = re.search(settings['parse_rtl']['reg_ex']["gpr"], line)
        if(match3):
            gpr.append(match3.group(0))
        else:
            gpr.append("")
        match4 = re.search(settings['parse_rtl']['reg_ex']["op1"], line)
        if(match4):
            op_0.append(match4.group(0))
        else:
            op_0.append("")
        match5 = re.search(settings['parse_rtl']['reg_ex']["op2"],line)
        if(match5):
            op_1.append(match5.group(0).split(", ")[2])
        else:
            op_1.append("")
        match6 = re.search(settings['parse_rtl']['reg_ex']["csr"],line)
        if(match6):
            csr.append(match6.group(0))
        else:
            print(line)
            raise RuntimeError("CSR RegEx not complete")

        match7 = re.findall(settings['parse_rtl']['reg_ex']['pc_csr_status_tval'],line)
        if(len(match7) == 3):
            print(match7)
            print(line)
        gpr_val.append(match7[1])
        status.append(match7[2])
        tval.append(match7[3])
        mode.append(re.search(settings['parse_rtl']['reg_ex']["mode"],line).group(0))
        exception.append(re.search(settings['parse_rtl']['reg_ex']["exception"],line).group(0))



columns = settings['parse_rtl']['columns']
rtl_log = pd.DataFrame(columns=columns)
rtl_log[columns[0]] = pc
rtl_log[columns[1]] = instr
rtl_log[columns[2]] = pd.Series(gpr) +":"+ pd.Series(gpr_val)
rtl_log[columns[3]] = pd.Series(csr) +":"+ pd.Series(status)
rtl_log[columns[4]] = binary
rtl_log[columns[5]] = mode
rtl_log[columns[6]] = instr_str
rtl_log[columns[7]] = pd.Series(gpr)+","+pd.Series(op_0)+","+pd.Series(op_1)
rtl_log[columns[8]] = valid
rtl_log[columns[9]] = exception
#log[columns[9]] = tval
del pc,binary,instr,instr_str,gpr,csr,gpr_val,op_0,op_1,status,tval,mode,exception,valid #delete tmp lists
rtl_log = rtl_log[boot_loader_length:].reset_index(drop=True) #boot_loader_length is where the bootloader ends
def replace_gpr(pd_series,gpr,re_gpr):
    tmp = []
    for line in tqdm(pd_series):
        split_line = line.split(":")
        if(len(split_line[0]) > 0 and split_line[0] == gpr):
            split_line[0] = re_gpr
        ln = split_line[0]+":"+split_line[1]
        tmp.append(ln)
    return pd.Series(tmp)
rtl_log.gpr = replace_gpr(rtl_log.gpr, 'fp','s0') #replace fp with s0 to keep convention consistent
rtl_log['valid'] = rtl_log['valid'].astype('int32') #not true when we have an exception (ecall for instance)
#rtl_log = rtl_log[rtl_log.valid == 1].reset_index(drop=True) #only keep valid instructions
rtl_log = rtl_log[~((rtl_log['instr'] == "addi") & (rtl_log.shift(1)['instr'] == "mret") & (rtl_log['valid'] == 0))].reset_index(drop=True) #deal with gaislers addi injection after mret, this removes all of those lines


100%|██████████| 100741/100741 [00:00<00:00, 834927.38it/s]


## Convert OVPsimlog to csv

In [5]:
logs = glob.glob(settings["gen_csv"]["paths"]["csv_log"])
csvs = glob.glob(settings["gen_csv"]["paths"]["csv_out"])
for log in logs:
    
    process = Popen(settings['gen_csv']['process']+log+ " --csv "+log.strip(".log")+".csv", shell=True, stdout=PIPE, stderr=PIPE)
    stdout, stderr = process.communicate()
csvs = glob.glob(settings["gen_csv"]["paths"]["csv_out"])
sim_log = pd.read_csv(csvs[0])
sim_log['mode'] = sim_log['mode'].astype('Int64')
sim_log['gpr'] = sim_log['gpr'].astype('str')
sim_log['csr'] = sim_log['csr'].astype('str')
#Mask LSB with 0 to contend with gaisler not supporting pmp_g = 0
#sim_log['gpr'] = sim_log[(sim_log.gpr != "nan") & sim_log.gpr.ne(rtl_log.gpr)].gpr.apply(lambda x: "{}:{:016x}".format(x.split(":")[0],int(x.split(":")[1],16) & int("FFFE",16)))
#sim_log['gpr'] = sim_log['gpr'].astype('str')


## Compare series to see if run was successful

In [6]:
#Truncation
if(abs(len(sim_log)-len(rtl_log) < 100)):
    if(len(sim_log) < len(rtl_log)):
        rtl_log=rtl_log.truncate(after=len(sim_log)-1)
        print("Truncating rtl_log to {} elements to meet size of sim_log".format(len(rtl_log)))
    elif(len(sim_log) == len(rtl_log)):
        print("Both logs are equally sized")

    else:
        sim_log=sim_log.truncate(after=len(rtl_log)-1)
        print("Truncating sim_log to {} elements to meet size of rtl_log".format(len(rtl_log)))
else:
    raise RuntimeError("Lengths of series differs with "+str(abs(len(sim_log)-len(rtl_log)))+" elements") 

#Csr match conditions
mt_csr_0 = sim_log.csr != "nan"

    
rtl_csr = rtl_log[(rtl_log.csr.ne(sim_log.csr) & mt_csr_0)].csr #these are the ones we can't exclude directly
sim_csr = sim_log[rtl_log.csr.ne(sim_log.csr) & mt_csr_0].csr.str.split(";") 
csr_match_count = compare_csr(sim_csr, rtl_csr)

mt_csr_1 = csr_information_present(sim_csr,rtl_csr) #ignore csr comparisons where for example mstatus is present the iss but not in the rtl log



matching_gpr = len(sim_log[(sim_log.gpr != "nan") & (rtl_log.gpr == sim_log.gpr)])
nan_cnt_gpr = len(sim_log[sim_log.gpr == "nan"])


rtl_csr = rtl_log[(rtl_log.csr.ne(sim_log.csr) & mt_csr_0)].csr #these are the ones we can't exclude directly
sim_csr = sim_log[rtl_log.csr.ne(sim_log.csr) & mt_csr_0].csr.str.split(";") 
csr_match_count = compare_csr(sim_csr, rtl_csr)

#these csrs only contain one update to the csr, however, where ovpsim reports more than one update we use csr_match_count
matching_csr = sim_log[((sim_log.csr != "nan")) & (rtl_log.csr.eq(sim_log.csr))].csr.count() + csr_match_count
nan_cnt_csr = sim_log[(sim_log.csr == "nan")].csr.count() + mt_csr_1[mt_csr_1 == False].count()


tot_cnt_gpr = matching_gpr + nan_cnt_gpr
gpr_match = (tot_cnt_gpr == len(rtl_log.gpr))
binary_match = rtl_log.binary.eq(sim_log.binary).all()
pc_match = rtl_log.pc.eq(sim_log.pc).all()

csr_match = (matching_csr+nan_cnt_csr == len(rtl_log.csr))

if(gpr_match and binary_match and pc_match and csr_match):
    print("GPR, Binary, PC and CSR contents match\nIgnored {:.1%} CSR and {:.1%} of GPR comparisons".format(nan_cnt_csr/rtl_log.csr.count(),nan_cnt_gpr/rtl_log.gpr.count()))
else:
    print("PC match: {}\nBinary match: {}\nGPR match: {}\nCSR match: {}\nIgnored {:.1%} CSR and {:.1%} of GPR comparisons".format(pc_match,binary_match,gpr_match,csr_match,nan_cnt_csr/rtl_log.csr.count(),nan_cnt_gpr/rtl_log.gpr.count()))


Truncating rtl_log to 99265 elements to meet size of sim_log
GPR, Binary, PC and CSR contents match
Ignored 98.5% CSR and 72.5% of GPR comparisons


In [28]:
columns = ["Test",'Iteration','Date','GPR Pass','CSR Pass','PC Pass','Binary Pass','Test Path','RTL Log Path','ISS Log Path','ISS Snippet','Number of Faults']
errors = pd.DataFrame(columns=columns)
# errors['Test'] = tests[test_number]
# errors['Iteration'] = settings['gen_test']['iteration']
# errors['Date'] = current_date
# errors['GPR Pass'] = gpr_match
# errors['PC Pass'] = pc_match
# errors['Binary Pass'] = binary_match
# errors['CSR Pass'] = csr_match


# "Test" = name of test,
# 'Iteration' = Which iteration failed,
# 'Date' = Date of Failure,
# 'GPR Pass' = Boolean,
# 'CSR Pass' = Boolean,
# 'PC Pass' += Boolean,
# 'Binary Pass' = Boolean,
# 'Test Path' = Full path to where asm files are located,
# 'RTL Log Path' = Full path to RTL trace,
# 'ISS Log Path' = Full path to ISS trace,
# 'ISS Snippet' = ISS Reference output for diverging trace,
# 'RTL Snippet' = RTL trace for the diverging lines
# rtl_log.to_csv('/home/jonathanjonsson/Documents/test_log.csv',mode='a')

'0'

In [ ]:
## Coverage results

path = glob.glob(settings["gen_cov"]["paths"]["logs"])
process = Popen(settings["gen_cov"]["process_param"]+settings["gen_cov"]["test_param"]+" --dir "+path[0], shell=True, stdout=PIPE, stderr=PIPE)

stdout, stderr = process.communicate()
cov_out = stdout.decode('ascii')
cov_err = stderr.decode('ascii')
if(find_success(settings['gen_cov']['test_sucess'],cov_err)):
    print("Coverage results produced successfully")
else:
    print(cov_err)
